# Chapter 8: Ray Tracing - Core Implementation

## Building a Complete Ray Tracer from Scratch

This notebook covers:
- Ray generation and camera model
- Ray-object intersection
- Shading and shadows
- Reflection and refraction
- Complete recursive ray tracer
- Anti-aliasing

**Key References:** Marschner & Shirley Ch. 4, 13, Gambetta Part II

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_08_ray_tracing_-_core_implementation.ipynb)

---

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Tuple, List, Optional
from dataclasses import dataclass

print("✓ Imports loaded")

---

## 1. Ray Tracing Theory

### 1.1 Ray Tracing Overview

**Ray tracing** simulates light transport by tracing rays from the camera through each pixel into the scene.

**Rendering equation** (simplified for ray tracing):
$$L_o(\mathbf{p}, \omega_o) = L_e(\mathbf{p}, \omega_o) + \int_{\Omega} f_r(\mathbf{p}, \omega_i, \omega_o) L_i(\mathbf{p}, \omega_i) (\mathbf{n} \cdot \omega_i) \, d\omega_i$$

**Algorithm:**
1. **Ray generation**: For each pixel, generate a ray from camera through pixel
2. **Intersection**: Find nearest object the ray hits
3. **Shading**: Compute color at hit point:
   - Local illumination (ambient, diffuse, specular)
   - Shadows (shadow rays to lights)
   - Reflection (recursive ray tracing)
   - Refraction (for transparent objects)
4. **Recursion**: Trace reflected/refracted rays up to max depth

### 1.2 Camera Model

**Pinhole camera** model with:
- **Eye position** $\mathbf{e}$: Camera location
- **Look-at point** $\mathbf{l}$: Where camera points
- **Up vector** $\mathbf{u}$: Camera orientation
- **Field of view** (FOV): Viewing angle
- **Image plane**: Virtual screen at distance $d$ from eye

**Camera coordinate system:**
$$\mathbf{w} = -\frac{\mathbf{l} - \mathbf{e}}{\|\mathbf{l} - \mathbf{e}\|} \quad \text{(backward)}$$
$$\mathbf{u} = \frac{\mathbf{up} \times \mathbf{w}}{\|\mathbf{up} \times \mathbf{w}\|} \quad \text{(right)}$$
$$\mathbf{v} = \mathbf{w} \times \mathbf{u} \quad \text{(up)}$$

**Ray through pixel** $(i, j)$:
$$\mathbf{d} = -d\mathbf{w} + (i - n_x/2)\frac{w_{\text{sensor}}}{n_x}\mathbf{u} + (j - n_y/2)\frac{h_{\text{sensor}}}{n_y}\mathbf{v}$$

where:
- $d$ = focal length (usually 1)
- $n_x, n_y$ = image resolution
- $w_{\text{sensor}}, h_{\text{sensor}}$ = sensor dimensions (from FOV)

### 1.3 Fresnel Equations

**Fresnel equations** determine reflection/transmission at dielectric interfaces.

**Schlick's approximation:**
$$R(\theta) = R_0 + (1 - R_0)(1 - \cos\theta)^5$$

where:
$$R_0 = \left(\frac{n_1 - n_2}{n_1 + n_2}\right)^2$$

- $R(\theta)$ = reflectance at angle $\theta$
- $R_0$ = reflectance at normal incidence
- $n_1, n_2$ = refractive indices

### 1.4 Snell's Law

**Snell's law** for refraction:
$$n_1 \sin\theta_1 = n_2 \sin\theta_2$$

**Vector form:**
$$\mathbf{t} = \eta \mathbf{v} + (\eta c_1 - c_2)\mathbf{n}$$

where:
- $\eta = n_1/n_2$ = index ratio
- $c_1 = -\mathbf{v} \cdot \mathbf{n}$
- $c_2 = \sqrt{1 - \eta^2(1 - c_1^2)}$

**Total internal reflection** occurs when $\eta^2(1 - c_1^2) > 1$.

### 1.5 Shadow Rays

To check if point $\mathbf{p}$ is in shadow from light $\mathbf{L}$:
1. Cast **shadow ray** from $\mathbf{p}$ toward $\mathbf{L}$
2. If ray hits any object before reaching $\mathbf{L}$, point is in shadow
3. Use small **epsilon offset** from surface to avoid self-intersection

### 1.6 Recursive Ray Tracing

**Whitted ray tracing** (1980) recursively traces:
- **Primary rays**: From camera through pixels
- **Shadow rays**: From hit points to lights
- **Reflection rays**: Mirror reflection direction
- **Refraction rays**: Transmitted through transparent objects

**Color computation:**
$$C = C_{\text{local}} + k_r C_{\text{reflect}} + k_t C_{\text{refract}}$$

where:
- $C_{\text{local}}$ = local illumination (Phong/Blinn-Phong)
- $C_{\text{reflect}}$ = color from reflected ray (recursive)
- $C_{\text{refract}}$ = color from refracted ray (recursive)
- $k_r, k_t$ = reflection/transmission coefficients (from Fresnel)

**Termination conditions:**
- Max recursion depth reached
- Ray doesn't hit any object (background color)
- Accumulated weight below threshold

---

## 2. Core Implementation

In [ ]:
class Vec3:
    """3D Vector class with all necessary operations"""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar):
        if isinstance(scalar, Vec3):
            return Vec3(self.x * scalar.x, self.y * scalar.y, self.z * scalar.z)
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __rmul__(self, scalar):
        return self.__mul__(scalar)
    
    def __truediv__(self, scalar):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def __neg__(self):
        return Vec3(-self.x, -self.y, -self.z)
    
    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self):
        return math.sqrt(self.x**2 + self.y**2 + self.z**2)
    
    def normalize(self):
        l = self.length()
        return self / l if l > 0 else Vec3(0, 0, 0)
    
    def clamp(self, min_val=0.0, max_val=1.0):
        return Vec3(
            max(min_val, min(max_val, self.x)),
            max(min_val, min(max_val, self.y)),
            max(min_val, min(max_val, self.z))
        )
    
    def to_array(self):
        return np.array([self.x, self.y, self.z])
    
    def __repr__(self):
        return f"Vec3({self.x:.3f}, {self.y:.3f}, {self.z:.3f})"

Color = Vec3

class Ray:
    """Ray with origin and direction"""
    def __init__(self, origin: Vec3, direction: Vec3):
        self.origin = origin
        self.direction = direction.normalize()
    
    def at(self, t: float) -> Vec3:
        return self.origin + self.direction * t

@dataclass
class HitRecord:
    """Information about ray-object intersection"""
    t: float  # Distance along ray
    point: Vec3  # Hit point
    normal: Vec3  # Surface normal
    material: 'Material'  # Material at hit point
    front_face: bool = True  # True if ray hits from outside

class Material:
    """Material with reflection and refraction properties"""
    def __init__(self,
                 ambient: Color = None,
                 diffuse: Color = None,
                 specular: Color = None,
                 shininess: float = 32.0,
                 reflectivity: float = 0.0,
                 transparency: float = 0.0,
                 refractive_index: float = 1.0):
        self.ambient = ambient if ambient else Color(0.1, 0.1, 0.1)
        self.diffuse = diffuse if diffuse else Color(0.8, 0.8, 0.8)
        self.specular = specular if specular else Color(1.0, 1.0, 1.0)
        self.shininess = shininess
        self.reflectivity = reflectivity  # 0 = no reflection, 1 = perfect mirror
        self.transparency = transparency  # 0 = opaque, 1 = fully transparent
        self.refractive_index = refractive_index  # e.g., 1.5 for glass

class Sphere:
    """Sphere primitive for ray tracing"""
    def __init__(self, center: Vec3, radius: float, material: Material):
        self.center = center
        self.radius = radius
        self.material = material
    
    def intersect(self, ray: Ray) -> Optional[HitRecord]:
        """Ray-sphere intersection using quadratic formula"""
        oc = ray.origin - self.center
        a = ray.direction.dot(ray.direction)
        b = 2.0 * oc.dot(ray.direction)
        c = oc.dot(oc) - self.radius * self.radius
        discriminant = b * b - 4 * a * c
        
        if discriminant < 0:
            return None
        
        # Find nearest positive t
        sqrt_d = math.sqrt(discriminant)
        t = (-b - sqrt_d) / (2 * a)
        
        if t < 0.001:  # Avoid self-intersection
            t = (-b + sqrt_d) / (2 * a)
            if t < 0.001:
                return None
        
        point = ray.at(t)
        normal = (point - self.center).normalize()
        
        # Determine if ray hits from outside or inside
        front_face = ray.direction.dot(normal) < 0
        if not front_face:
            normal = -normal
        
        return HitRecord(t, point, normal, self.material, front_face)

class Plane:
    """Infinite plane for ray tracing"""
    def __init__(self, point: Vec3, normal: Vec3, material: Material):
        self.point = point
        self.normal = normal.normalize()
        self.material = material
    
    def intersect(self, ray: Ray) -> Optional[HitRecord]:
        """Ray-plane intersection"""
        denom = self.normal.dot(ray.direction)
        
        if abs(denom) < 1e-6:
            return None  # Parallel
        
        t = (self.point - ray.origin).dot(self.normal) / denom
        
        if t < 0.001:
            return None
        
        point = ray.at(t)
        normal = self.normal
        front_face = ray.direction.dot(normal) < 0
        if not front_face:
            normal = -normal
        
        return HitRecord(t, point, normal, self.material, front_face)

class Light:
    """Point light source"""
    def __init__(self, position: Vec3, color: Color, intensity: float = 1.0):
        self.position = position
        self.color = color
        self.intensity = intensity

print("✓ Core classes loaded")

---

## 3. Ray Tracer Implementation

In [ ]:
class Camera:
    """Pinhole camera for ray generation"""
    def __init__(self,
                 eye: Vec3,
                 look_at: Vec3,
                 up: Vec3,
                 fov: float,  # in degrees
                 aspect_ratio: float):
        self.eye = eye
        
        # Build camera coordinate system
        self.w = (eye - look_at).normalize()  # Backward
        self.u = up.cross(self.w).normalize()  # Right
        self.v = self.w.cross(self.u)  # Up
        
        # Compute image plane dimensions
        theta = fov * math.pi / 180.0
        half_height = math.tan(theta / 2.0)
        half_width = aspect_ratio * half_height
        
        self.horizontal = self.u * (2.0 * half_width)
        self.vertical = self.v * (2.0 * half_height)
        self.lower_left = self.eye - self.u * half_width - self.v * half_height - self.w
    
    def get_ray(self, u: float, v: float) -> Ray:
        """Generate ray through normalized coordinates (u, v) in [0, 1]"""
        direction = self.lower_left + self.horizontal * u + self.vertical * v - self.eye
        return Ray(self.eye, direction)

class Scene:
    """Scene containing objects and lights"""
    def __init__(self):
        self.objects = []
        self.lights = []
        self.background_color = Color(0.5, 0.7, 1.0)  # Sky blue
    
    def add_object(self, obj):
        self.objects.append(obj)
    
    def add_light(self, light: Light):
        self.lights.append(light)
    
    def intersect(self, ray: Ray) -> Optional[HitRecord]:
        """Find nearest intersection of ray with scene"""
        closest_hit = None
        closest_t = float('inf')
        
        for obj in self.objects:
            hit = obj.intersect(ray)
            if hit and hit.t < closest_t:
                closest_hit = hit
                closest_t = hit.t
        
        return closest_hit

def reflect(v: Vec3, n: Vec3) -> Vec3:
    """Reflect vector v about normal n"""
    return v - n * (2.0 * v.dot(n))

def refract(v: Vec3, n: Vec3, eta: float) -> Optional[Vec3]:
    """Refract vector v through interface with normal n and index ratio eta"""
    cos_theta = min(-v.dot(n), 1.0)
    r_out_perp = (v + n * cos_theta) * eta
    r_out_parallel_squared = 1.0 - r_out_perp.dot(r_out_perp)
    
    if r_out_parallel_squared < 0:
        return None  # Total internal reflection
    
    r_out_parallel = n * (-math.sqrt(r_out_parallel_squared))
    return r_out_perp + r_out_parallel

def schlick(cosine: float, ref_idx: float) -> float:
    """Schlick's approximation for Fresnel reflectance"""
    r0 = ((1 - ref_idx) / (1 + ref_idx)) ** 2
    return r0 + (1 - r0) * ((1 - cosine) ** 5)

def compute_local_illumination(hit: HitRecord, scene: Scene, view_dir: Vec3) -> Color:
    """Compute local illumination using Blinn-Phong"""
    color = hit.material.ambient * Color(0.1, 0.1, 0.1)  # Ambient
    
    for light in scene.lights:
        # Direction to light
        light_dir = (light.position - hit.point).normalize()
        distance_to_light = (light.position - hit.point).length()
        
        # Check shadow
        shadow_ray = Ray(hit.point + hit.normal * 0.001, light_dir)
        shadow_hit = scene.intersect(shadow_ray)
        
        if shadow_hit and shadow_hit.t < distance_to_light:
            continue  # In shadow
        
        # Diffuse
        n_dot_l = max(0.0, hit.normal.dot(light_dir))
        diffuse = hit.material.diffuse * light.color * (light.intensity * n_dot_l)
        
        # Specular (Blinn-Phong)
        halfway = (light_dir + view_dir).normalize()
        n_dot_h = max(0.0, hit.normal.dot(halfway))
        spec_factor = pow(n_dot_h, hit.material.shininess)
        specular = hit.material.specular * light.color * (light.intensity * spec_factor)
        
        color = color + diffuse + specular
    
    return color.clamp(0.0, 1.0)

def trace_ray(ray: Ray, scene: Scene, depth: int, max_depth: int = 5) -> Color:
    """Recursive ray tracing with reflection and refraction"""
    if depth >= max_depth:
        return Color(0, 0, 0)
    
    hit = scene.intersect(ray)
    
    if not hit:
        return scene.background_color
    
    # Local illumination
    view_dir = -ray.direction
    local_color = compute_local_illumination(hit, scene, view_dir)
    
    # Initialize final color
    final_color = local_color * (1.0 - hit.material.reflectivity - hit.material.transparency)
    
    # Reflection
    if hit.material.reflectivity > 0:
        reflect_dir = reflect(ray.direction, hit.normal)
        reflect_ray = Ray(hit.point + hit.normal * 0.001, reflect_dir)
        reflect_color = trace_ray(reflect_ray, scene, depth + 1, max_depth)
        final_color = final_color + reflect_color * hit.material.reflectivity
    
    # Refraction
    if hit.material.transparency > 0:
        eta = 1.0 / hit.material.refractive_index if hit.front_face else hit.material.refractive_index
        
        refract_dir = refract(ray.direction, hit.normal, eta)
        
        if refract_dir:
            # Use Fresnel to blend reflection and refraction
            cos_theta = min(-ray.direction.dot(hit.normal), 1.0)
            fresnel = schlick(cos_theta, hit.material.refractive_index)
            
            refract_ray = Ray(hit.point - hit.normal * 0.001, refract_dir)
            refract_color = trace_ray(refract_ray, scene, depth + 1, max_depth)
            
            # Blend reflection and refraction based on Fresnel
            final_color = final_color + refract_color * hit.material.transparency * (1 - fresnel)
        else:
            # Total internal reflection
            reflect_dir = reflect(ray.direction, hit.normal)
            reflect_ray = Ray(hit.point + hit.normal * 0.001, reflect_dir)
            reflect_color = trace_ray(reflect_ray, scene, depth + 1, max_depth)
            final_color = final_color + reflect_color * hit.material.transparency
    
    return final_color.clamp(0.0, 1.0)

def render(scene: Scene, camera: Camera, width: int, height: int, samples_per_pixel: int = 1) -> np.ndarray:
    """Render scene to image"""
    image = np.zeros((height, width, 3))
    
    for j in range(height):
        if j % 50 == 0:
            print(f"Rendering line {j}/{height}...")
        
        for i in range(width):
            color = Color(0, 0, 0)
            
            # Anti-aliasing: multiple samples per pixel
            for _ in range(samples_per_pixel):
                u = (i + np.random.random()) / width
                v = 1.0 - (j + np.random.random()) / height  # Flip v
                
                ray = camera.get_ray(u, v)
                color = color + trace_ray(ray, scene, 0)
            
            color = color / samples_per_pixel
            image[j, i] = color.to_array()
    
    print("Rendering complete!")
    return image

print("✓ Ray tracer implementation loaded")

---

## 4. Example Scenes

In [ ]:
# Example 1: Basic Scene with Shadows
print("Example 1: Basic Scene with Shadows\n")

scene = Scene()

# Materials
red_diffuse = Material(
    ambient=Color(0.2, 0.0, 0.0),
    diffuse=Color(0.8, 0.0, 0.0),
    specular=Color(1.0, 1.0, 1.0),
    shininess=32.0
)

blue_diffuse = Material(
    ambient=Color(0.0, 0.0, 0.2),
    diffuse=Color(0.0, 0.0, 0.8),
    specular=Color(1.0, 1.0, 1.0),
    shininess=32.0
)

ground = Material(
    ambient=Color(0.1, 0.1, 0.1),
    diffuse=Color(0.5, 0.5, 0.5),
    specular=Color(0.3, 0.3, 0.3),
    shininess=10.0
)

# Objects
scene.add_object(Sphere(Vec3(0, 0, -3), 0.5, red_diffuse))
scene.add_object(Sphere(Vec3(-1.2, 0, -3.5), 0.6, blue_diffuse))
scene.add_object(Plane(Vec3(0, -0.5, 0), Vec3(0, 1, 0), ground))

# Lights
scene.add_light(Light(Vec3(2, 3, -2), Color(1, 1, 1), intensity=8.0))

# Camera
camera = Camera(
    eye=Vec3(0, 0.5, 0),
    look_at=Vec3(0, 0, -3),
    up=Vec3(0, 1, 0),
    fov=60.0,
    aspect_ratio=16.0/9.0
)

# Render
image = render(scene, camera, 400, 225, samples_per_pixel=1)

plt.figure(figsize=(12, 7))
plt.imshow(image)
plt.title("Basic Ray Tracing with Shadows")
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Example 2: Reflective Spheres
print("\nExample 2: Reflective Spheres\n")

scene = Scene()
scene.background_color = Color(0.1, 0.1, 0.2)

# Materials
mirror = Material(
    ambient=Color(0.1, 0.1, 0.1),
    diffuse=Color(0.2, 0.2, 0.2),
    specular=Color(1.0, 1.0, 1.0),
    shininess=100.0,
    reflectivity=0.9
)

colored_mirror = Material(
    ambient=Color(0.2, 0.1, 0.0),
    diffuse=Color(0.6, 0.3, 0.1),
    specular=Color(1.0, 1.0, 1.0),
    shininess=100.0,
    reflectivity=0.6
)

ground = Material(
    ambient=Color(0.05, 0.05, 0.05),
    diffuse=Color(0.3, 0.3, 0.3),
    specular=Color(0.5, 0.5, 0.5),
    shininess=50.0,
    reflectivity=0.3
)

# Objects
scene.add_object(Sphere(Vec3(0, 0, -4), 0.7, mirror))
scene.add_object(Sphere(Vec3(-1.5, -0.3, -3), 0.4, colored_mirror))
scene.add_object(Sphere(Vec3(1.2, -0.2, -3.5), 0.5, colored_mirror))
scene.add_object(Plane(Vec3(0, -0.7, 0), Vec3(0, 1, 0), ground))

# Lights
scene.add_light(Light(Vec3(3, 4, -1), Color(1.0, 0.9, 0.8), intensity=10.0))
scene.add_light(Light(Vec3(-2, 2, -2), Color(0.5, 0.5, 0.7), intensity=5.0))

# Camera
camera = Camera(
    eye=Vec3(0, 0.3, 0),
    look_at=Vec3(0, 0, -4),
    up=Vec3(0, 1, 0),
    fov=60.0,
    aspect_ratio=16.0/9.0
)

# Render
image = render(scene, camera, 400, 225, samples_per_pixel=1)

plt.figure(figsize=(12, 7))
plt.imshow(image)
plt.title("Reflective Spheres")
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Example 3: Glass Sphere with Refraction
print("\nExample 3: Glass Sphere with Refraction\n")

scene = Scene()
scene.background_color = Color(0.5, 0.7, 1.0)

# Materials
glass = Material(
    ambient=Color(0.0, 0.0, 0.0),
    diffuse=Color(0.1, 0.1, 0.1),
    specular=Color(1.0, 1.0, 1.0),
    shininess=100.0,
    reflectivity=0.1,
    transparency=0.9,
    refractive_index=1.5
)

red_matte = Material(
    ambient=Color(0.2, 0.0, 0.0),
    diffuse=Color(0.8, 0.0, 0.0),
    specular=Color(0.5, 0.5, 0.5),
    shininess=32.0
)

ground = Material(
    ambient=Color(0.1, 0.1, 0.1),
    diffuse=Color(0.6, 0.6, 0.6),
    specular=Color(0.2, 0.2, 0.2),
    shininess=10.0
)

# Objects
scene.add_object(Sphere(Vec3(0, 0, -3), 0.6, glass))
scene.add_object(Sphere(Vec3(-1.5, -0.3, -2.5), 0.3, red_matte))
scene.add_object(Plane(Vec3(0, -0.6, 0), Vec3(0, 1, 0), ground))

# Lights
scene.add_light(Light(Vec3(2, 3, -1), Color(1, 1, 1), intensity=10.0))

# Camera
camera = Camera(
    eye=Vec3(0, 0.2, 0),
    look_at=Vec3(0, 0, -3),
    up=Vec3(0, 1, 0),
    fov=60.0,
    aspect_ratio=16.0/9.0
)

# Render
image = render(scene, camera, 400, 225, samples_per_pixel=2)

plt.figure(figsize=(12, 7))
plt.imshow(image)
plt.title("Glass Sphere with Refraction")
plt.axis('off')
plt.tight_layout()
plt.show()

---

## Summary

In this chapter, you implemented:

✅ **Complete Ray Tracer** - From scratch implementation  
✅ **Camera Model** - Pinhole camera with FOV  
✅ **Ray-Object Intersection** - Spheres and planes  
✅ **Local Illumination** - Blinn-Phong shading model  
✅ **Shadows** - Shadow ray casting  
✅ **Reflection** - Recursive mirror reflections  
✅ **Refraction** - Glass materials with Snell's law  
✅ **Fresnel Effects** - Schlick's approximation  
✅ **Anti-aliasing** - Multiple samples per pixel  

**Key Insights:**
- Ray tracing simulates light paths backwards from camera
- Recursion enables realistic reflection and refraction
- Shadow rays determine if points are illuminated
- Fresnel equations blend reflection and refraction
- Anti-aliasing reduces jagged edges
- Small epsilon offsets prevent self-intersection artifacts

**Performance Notes:**
- Ray tracing is computationally expensive (O(rays × objects))
- Each pixel requires intersection tests with all objects
- Recursion depth affects quality and performance
- Anti-aliasing multiplies rendering time by samples per pixel

**Next Chapter:** Acceleration Structures (BVH, KD-Trees)